In [19]:
from astropy.io import fits
from astropy.io.fits import getdata
from astropy.table import Table
import pylab as P
import numpy as np
import seaborn
import itertools
import operator

hdulist = fits.open('../2004B.mask.0.36.01.nn/ccd19.fits')
hdulistE = fits.open('../2004B.mask.0.36.01_enlarged.nn/ccd19.fits')

d = hdulist[1].data
dE = hdulistE[1].data

print d

[(   0,    0, 2048,    1) (1108, 1482,    6,    9) (1924, 1569,    6,    6)
 (1888, 1643,    4,    5) (2002, 2686,    5,    6) (2019, 3042,    1,  101)
 ( 459, 3734,   11,    8) (2039,  211,    1, 4401) (2038,  226,    1,   41)
 (2040,  226,    1,   41) ( 324, 2179,    2, 2433) (1434, 3535,    2, 1077)
 (   0, 4611,  324,    1) ( 326, 4611, 1108,    1) (1436, 4611,  603,    1)
 (2040, 4611,    8,    1)]


In [151]:
print du[94:99,1492:1497]
for l in du.shape[0]:
    for c in du.shape[1]:
        if du[l, c] == 0 :
    

[[1 1 1 1 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 1 1 1 1]]


In [9]:
import lsst.afw.image as afwImage
import lsst.afw.detection as afwDetect
import lsst.afw.geom as afwGeom
import lsst.afw.display.ds9 as ds9
from astropy.io import fits

ccd = 1
hdulist = fits.open('../2004B.mask.0.36.01.nn/ccd' + str("%02i"%ccd) + '.fits')
d = hdulist[1].data
hdulistE = fits.open('../2004B.mask.0.36.01_enlarged.nn/ccd' + str("%02i"%ccd) + '.fits')
dE = hdulistE[1].data

bb = afwGeom.Box2I(afwGeom.Point2I(32, 0), afwGeom.Point2I(2079, 4611))
im = afwImage.ImageF('../2004B.mask.0.36.01.fits[' + str(ccd+1) + ']', bbox=bb, 
                           origin=afwImage.ImageOrigin.PARENT)
print im.getXY0()
im *= -1.
im += 1.
im *= 10.
#ds9.mtv(im, frame=1)

level = 2
s = afwDetect.FootprintSet(im, afwDetect.Threshold(level, polarity=True))

x0R = []
y0R = []
width = []
height= []

x0E = []
y0E = []
widthE = []
heightE= []

count = 1
for f in s.getFootprints():
    fpl = afwDetect.footprintToBBoxList(f)
    for i in fpl:
        i.shift(afwGeom.Extent2I(-32,0))
        x0 = i.getBeginX()
        y0 = i.getBeginY()
        w0 = i.getWidth()
        h0 = i.getHeight()
        
        x0R.append(x0)
        y0R.append(y0)
        width.append(w0)
        height.append(h0)
        
        if (x0 % 2 == 0) :
            x1 = x0
            if (w0 % 2 == 0) :
                w1 = w0
            else :
                w1 = w0 + 1
        else :
            x1 = max(x0 - 1, 0)
            if (w0 % 2 == 0) :
                w1 = min(w0 + 2, 2048)
            else :
                w1 = min(w0 + 1, 2048)
        if (y0 % 2 == 0):
            y1 = y0
            if (h0 % 2 == 0) :
                h1 = h0
            else :
                h1 = min(h0 + 1, 4612)
        else :
            y1 = max(y0 - 1, 0)
            if (h0 % 2 == 0) :
                h1 = min(h0 + 2, 4612)
            else :
                h1 = min(h0 + 1, 4612)
        
        o = dE[count-1]
        if x1 != o[0] or y1 != o[1] or w1 != o[2] or h1 != o[3] :
            st = "<==="
        else :
            st = ""
            
        print "%2i : %5i %5i %5i %5i : %5i %5i %5i %5i : %5i %5i %5i %5i : %5i %5i %5i %5i %s"%(count, x0, y0, w0, h0,
            d[count-1][0], d[count-1][1], d[count-1][2], d[count-1][3],
            x1, y1, w1, h1, dE[count-1][0], dE[count-1][1], dE[count-1][2], dE[count-1][3], st)
        count += 1

(32, 0)
 1 :     0     0  2048     1 :     0     0  2048     1 :     0     0  2048     2 :     0     0  2048     2 
 2 :  1655   626     1   101 :  1655   626     1   101 :  1654   626     2   102 :  1654   626     2   102 
 3 :   839   835     1   101 :   839   835     1   101 :   838   834     2   102 :   838   834     2   102 
 4 :   994  2922     1   101 :   994  2922     1   101 :   994  2922     2   102 :   994  2922     2   102 
 5 :   892   555     2  4057 :   892   555     2  4057 :   892   554     2  4058 :   892   554     2  4058 
 6 :   848  1649     1  2963 :   848  1649     1  2963 :   848  1648     2  2964 :   848  1648     2  2964 
 7 :   788  3526     1  1086 :   788  3526     1  1086 :   788  3526     2  1086 :   788  3526     2  1086 
 8 :    56  3979     1   633 :    56  3979     1   633 :    56  3978     2   634 :    56  3978     2   634 
 9 :     0  4611    56     1 :     0  4611    56     1 :     0  4610    56     2 :     0  4610    56     2 
10 :    57  4611   7

In [57]:
print '%5i' %dE[count-2][0]

  894


In [86]:
from astropy.io import fits
import numpy as np

col1 = fits.Column(name="x0", format="I", array=np.asarray(x0R))
col2 = fits.Column(name="y0", format="I", array=np.asarray(y0R))
col3 = fits.Column(name="width", format="I", array=np.asarray(width))
col4 = fits.Column(name="height", format="I", array=np.asarray(height))

tbhdr = fits.Header()
tbhdr['SERIAL'] = hdulist[1].header['SERIAL']
tbhdr['CDATE'] = hdulist[1].header['CDATE']

cols = fits.ColDefs([col1, col2, col3, col4])
tbhdu = fits.BinTableHDU.from_columns(cols, header=tbhdr)

#tbhdu = fits.TableHDU(header=prihdr)

#thdulist = fits.HDUList([prihdu, tbhdu])
tbhdu.writeto('test.fits', overwrite=True)

In [75]:
print hdulist[1].header['SERIAL'], hdulist[1].header['CDATE']

 835234 Fri May 16 00:29:53 2014


In [72]:
fits.

XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                    8 / length of dimension 1                          NAXIS2  =                   13 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                    4 / number of table fields                         TTYPE1  = 'x0      '                                                            TFORM1  = 'I       '                                                            TTYPE2  = 'y0      '                                                            TFORM2  = 'I       '                                                            TTYPE3  = 'width   '                    

In [11]:
import os

a = os.path.join('/home/boutigny/LSST/my_packages/obs_cfht/megacam', 'defects', 'test', '.fits')
print a

/home/boutigny/LSST/my_packages/obs_cfht/megacam/defects/test/.fits
